In [ ]:
import pickle
def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    # print(f"Data saved to {filename}")
    
# 定义一个函数，用于加载文件中的数据
# 定义一个函数，用于加载文件中的数据
    # 打开文件，以二进制模式读取
def load_data(filename):
        # 使用pickle模块加载文件中的数据
    with open(filename, 'rb') as f:
    # 返回加载的数据
        data = pickle.load(f)
    return data
list_gene_all=load_data('./data/list_gene_all.pkl')
#与07ig.py配合使用,统计中间结果
gene_list_choose_ig=['NFKBIA', 'TNFAIP3', 'FOS', 'FOSB', "KLF6"]
content_list_index=[list_gene_all.index(i) for i in choose_gene_name]



In [ ]:
fen=load_data('./data/fen.pkl')
import scanpy as sc
adata=sc.read_h5ad('./data/adata_process.h5ad',backed='r')
adata2=adata[fen['test']]
df2=adata2.obs.copy()
df3=df2.loc[:,['celltype_L3','stage']]
celltypel3_list=df3['celltype_L3'].unique().tolist()

dict_HDs={i:[] for i in celltypel3_list}
dict_IRs={i:[] for i in celltypel3_list}
dict_INRs={i:[] for i in celltypel3_list}
iter=0
from tqdm import tqdm
for index, row in tqdm(df3.iterrows()):
    celltype=row['celltype_L3']
    stage=row['stage']
    if stage=='IRs':
        dict_IRs[celltype].append(iter)
    elif stage=='INRs':
        dict_INRs[celltype].append(iter)
    elif stage=='HDs':
        dict_HDs[celltype].append(iter)
    iter+=1

save_data(dict_IRs,'./data/dict_IRs.pkl')
save_data(dict_INRs,'./data/dict_INRs.pkl')
save_data(dict_HDs,'./data/dict_HDs.pkl')

In [ ]:

from tqdm import tqdm
def get_data(gene_name='APOBEC3A',base_path='./ig/ig_output'):

    # gene的文件存储名,是用gene在582个gene的位置确定的
    file_name=list_gene_all.index(gene_name)

    if os.path.exists(base_path+'/'+str(file_name)+'.pkl'):
        list_i=load_data(base_path+'/'+str(file_name)+'.pkl')
        data=np.concatenate(list_i,axis=0)

    else:
        return None
    return data

In [ ]:
#输入基因
base_path='./ig/ig_output'
path_base2='./ig/all'
path_base1='./ig/l3'

dict_HDs=load_data('/home/share/huadjyin/home/zhouxuanchi/HIV/atac_to_gene_new_data_0218/data/ig_tianyu_need_l3_again/data/dict_hds.pkl')
dict_INRs=load_data('/home/share/huadjyin/home/zhouxuanchi/HIV/atac_to_gene_new_data_0218/data/ig_tianyu_need_l3_again/data/dict_inrs.pkl')
dict_IRs=load_data('/home/share/huadjyin/home/zhouxuanchi/HIV/atac_to_gene_new_data_0218/data/ig_tianyu_need_l3_again/data/dict_irs.pkl')
peak_name=load_data('./data/list_peak.pkl')

for index in range(len(gene_list_choose_ig)):
    gene=gene_list_choose_ig[index]
    gene_index=content_list_index[index]

    list_i=load_data(base_path+'/'+str(gene_index)+'.pkl')
    data=np.concatenate(list_i,axis=0)

    df=pd.DataFrame(columns=['rna_stage','celltype-l3']+peak_name)
    for i in dict_IRs.keys():
        list_choose=dict_IRs[i]
        df.loc[len(df)]=['IRs']+[i]+np.mean(np.abs(data[list_choose,:]), axis=0).tolist()
    for i in dict_INRs.keys():
        list_choose=dict_INRs[i]
        df.loc[len(df)]=['INRs']+[i]+np.mean(np.abs(data[list_choose,:]), axis=0).tolist()
    for i in dict_HDs.keys():
        list_choose=dict_HDs[i]
        df.loc[len(df)]=['HDs']+[i]+np.mean(np.abs(data[list_choose,:]), axis=0).tolist()
    df.to_csv(path_base1+'/'+str(gene)+'.csv')
    df=pd.DataFrame(columns=peak_name)
    df.loc[len(df)]=np.mean(np.abs(data), axis=0).tolist()
    df.to_csv(path_base2+'/'+str(gene)+'.csv')